# Class imbalance handling techniques and approaches, Analysis
 - fraud transaction data is used which is taken from kaggle (https://www.kaggle.com/c/ieee-fraud-detection)

In [1]:
# import basic libraries

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns



import warnings
warnings.filterwarnings("ignore")


In [2]:
# import data 


df_train_original = pd.read_csv("train_transaction.csv")

df_train = df_train_original.copy()

df_train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# check shape of df_train

print(df_train.shape)


(590540, 394)


In [4]:
# sample 25% from original

df_sample = df_train.sample(frac = 0.25)

print(df_sample.shape)
df_sample.head()

(147635, 394)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
80181,3067181,0,1726397,59.0,W,7728,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157972,3144972,0,3286141,59.0,W,2146,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320144,3307144,1,7952142,29.0,W,7207,111.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208506,3195506,0,4817694,49.0,W,5566,543.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291575,3278575,0,7177206,50.0,R,6019,583.0,150.0,visa,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_sample['isFraud'].unique()

array([0, 1])

In [6]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147635 entries, 80181 to 279946
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 444.9+ MB


In [7]:
df_sample.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,1.476350e+05,147635.000000,1.476350e+05,147635.000000,147635.000000,145376.000000,147226.000000,146531.00000,131357.000000,131357.000000,...,20662.000000,20662.000000,20662.000000,20662.000000,20662.000000,20662.000000,20662.000000,20662.000000,20662.000000,20662.000000
mean,3.282770e+06,0.035256,7.385887e+06,135.478889,9904.608148,362.319303,153.175200,199.19211,290.644998,86.796646,...,0.809893,759.295724,1438.023389,1059.978862,8.665376,59.234344,27.889362,54.894359,156.318471,102.671535
std,1.707066e+05,0.184426,4.625031e+06,233.524382,4908.182023,157.538620,11.281994,41.29142,101.474841,2.716272,...,4.840535,6395.654235,11443.662304,8167.907147,78.561105,299.245209,152.342506,538.365021,1041.398755,720.880594
min,2.987007e+06,0.000000,8.652900e+04,0.424000,1001.000000,100.000000,100.000000,100.00000,100.000000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.134716e+06,0.000000,3.028095e+06,43.382000,6019.000000,215.000000,150.000000,166.00000,204.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.282966e+06,0.000000,7.319425e+06,68.500000,9692.000000,361.000000,150.000000,226.00000,299.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.430702e+06,0.000000,1.128980e+07,125.000000,14259.000000,512.000000,150.000000,226.00000,330.000000,87.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.577538e+06,1.000000,1.581109e+07,6450.970000,18395.000000,600.000000,231.000000,237.00000,540.000000,102.000000,...,54.000000,108800.000000,146211.000000,108800.000000,4250.000000,6375.000000,6375.000000,51200.000000,51200.000000,51200.000000


In [8]:
sns.distplot(df_sample['idfraud'])

KeyError: 'idfraud'